In [1]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Setup complete!")

Setup complete!


/home/vulcan/miniconda3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cell 2: Load Data
print("Loading cleaned_aadhaar_dataset.csv...")
try:
    df = pd.read_csv('/home/vulcan/Abhay/Projects/ADA/Dataset/cleaned_aadhaar_dataset.csv')
    print(f"✓ Data loaded successfully: {df.shape}")
except FileNotFoundError:
    print("Error: 'cleaned_aadhaar_dataset.csv' not found.")
    raise

df['date'] = pd.to_datetime(df['date'])

# Sort by date for time-series features (CRITICAL for causality)
df = df.sort_values(['state', 'district', 'pincode', 'date']).reset_index(drop=True)
print(f"✓ Data sorted by geography and date")

Loading cleaned_aadhaar_dataset.csv...
✓ Data loaded successfully: (219091, 7)
✓ Data sorted by geography and date


In [3]:
# Cell 3: Temporal Features
print("Creating temporal features...")
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week.astype('int64')
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['days_in_month'] = df['date'].dt.days_in_month
df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

print(f"✓ Created {15} temporal features")

Creating temporal features...
✓ Created 15 temporal features


In [4]:
# Cell 4: Demographic Features
print("Creating demographic features...")
df['total_enrollments'] = df['age_0_5'] + df['age_5_17'] + df['age_18_greater']
df['child_ratio'] = df['age_0_5'] / (df['total_enrollments'] + 1e-10)
df['youth_ratio'] = df['age_5_17'] / (df['total_enrollments'] + 1e-10)
df['adult_ratio'] = df['age_18_greater'] / (df['total_enrollments'] + 1e-10)
df['child_adult_ratio'] = df['age_0_5'] / (df['age_18_greater'] + 1e-10)
df['youth_adult_ratio'] = df['age_5_17'] / (df['age_18_greater'] + 1e-10)
df['dependent_ratio'] = (df['age_0_5'] + df['age_5_17']) / (df['age_18_greater'] + 1e-10)

print(f"✓ Created {7} demographic features")

Creating demographic features...
✓ Created 7 demographic features


In [5]:
# Cell 5: Lag Features (Strictly Causal)
print("Creating lag features...")
group_cols = ['state', 'district', 'pincode']
lag_features_created = 0

for col in ['age_0_5', 'age_5_17', 'age_18_greater', 'total_enrollments']:
    df[f'{col}_lag1'] = df.groupby(group_cols)[col].shift(1)
    df[f'{col}_lag7'] = df.groupby(group_cols)[col].shift(7)
    df[f'{col}_lag30'] = df.groupby(group_cols)[col].shift(30)
    
    df[f'{col}_pct_change_1d'] = df.groupby(group_cols)[col].pct_change(1)
    df[f'{col}_pct_change_7d'] = df.groupby(group_cols)[col].pct_change(7)
    
    df[f'{col}_rolling_mean_7d'] = df.groupby(group_cols)[col].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )
    df[f'{col}_rolling_std_7d'] = df.groupby(group_cols)[col].transform(
        lambda x: x.rolling(7, min_periods=1).std()
    )
    df[f'{col}_rolling_mean_30d'] = df.groupby(group_cols)[col].transform(
        lambda x: x.rolling(30, min_periods=1).mean()
    )
    lag_features_created += 8

print(f"✓ Created {lag_features_created} lag/rolling features")

Creating lag features...
✓ Created 32 lag/rolling features


In [6]:
# Cell 6: Spatial Features
print("Creating spatial features...")

# State-level aggregates
state_stats = df.groupby(['state', 'date']).agg({
    'total_enrollments': ['sum', 'mean', 'std', 'count']
}).reset_index()
state_stats.columns = ['state', 'date', 'state_total', 'state_mean', 'state_std', 'state_count']
df = df.merge(state_stats, on=['state', 'date'], how='left')

# District-level aggregates
district_stats = df.groupby(['state', 'district', 'date']).agg({
    'total_enrollments': ['sum', 'mean', 'count']
}).reset_index()
district_stats.columns = ['state', 'district', 'date', 'district_total', 'district_mean', 'district_count']
df = df.merge(district_stats, on=['state', 'district', 'date'], how='left')

# Ratio features
df['pincode_vs_state_ratio'] = df['total_enrollments'] / (df['state_mean'] + 1e-10)
df['pincode_vs_district_ratio'] = df['total_enrollments'] / (df['district_mean'] + 1e-10)
df['district_vs_state_ratio'] = df['district_mean'] / (df['state_mean'] + 1e-10)

print(f"✓ Created {10} spatial features")

Creating spatial features...
✓ Created 10 spatial features


In [7]:
# Cell 7: Anomaly Detection Features
print("Creating anomaly features...")
df['z_score_state'] = (df['total_enrollments'] - df['state_mean']) / (df['state_std'] + 1e-10)
df['z_score_rolling'] = (df['total_enrollments'] - df['total_enrollments_rolling_mean_7d']) / (df['total_enrollments_rolling_std_7d'] + 1e-10)
df['enrollment_volatility'] = df['total_enrollments_rolling_std_7d'] / (df['total_enrollments_rolling_mean_7d'] + 1e-10)
df['is_spike'] = (abs(df['z_score_rolling']) > 2).astype(int)

print(f"✓ Created {4} anomaly detection features")

Creating anomaly features...
✓ Created 4 anomaly detection features


In [8]:
# Cell 8: Interaction Features
print("Creating interaction features...")
df['child_youth_interaction'] = df['age_0_5'] * df['age_5_17']

print(f"✓ Created {1} interaction feature")

Creating interaction features...
✓ Created 1 interaction feature


In [9]:
# Cell 9: Feature Engineering Summary
print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"Total shape: {df.shape}")
print(f"Total features: {len(df.columns)}")

print("\nNaN Summary:")
print(f"  - Lag features: {df[[c for c in df.columns if '_lag' in c]].isna().sum().sum():,} NaNs")
print(f"  - Rolling features: {df[[c for c in df.columns if '_rolling_' in c]].isna().sum().sum():,} NaNs")
print(f"  - Pct change: {df[[c for c in df.columns if '_pct_change_' in c]].isna().sum().sum():,} NaNs")
print("  (AutoGluon will handle NaN imputation)")


FEATURE ENGINEERING COMPLETE
Total shape: (219091, 77)
Total features: 77

NaN Summary:
  - Lag features: 1,463,288 NaNs
  - Rolling features: 90,292 NaNs
  - Pct change: 983,398 NaNs
  (AutoGluon will handle NaN imputation)


In [14]:
# Cell 10: Target Variable Creation
print("\n" + "="*60)
print("CREATING TARGET VARIABLES (LEAK-FREE)")
print("="*60)

# Compute train/test cutoff BEFORE creating targets
TRAIN_TEST_CUTOFF = df['date'].quantile(0.8)
train_mask = df['date'] <= TRAIN_TEST_CUTOFF
print(f"Train/Test cutoff date: {TRAIN_TEST_CUTOFF.date()}")
print(f"  Training samples: {train_mask.sum():,}")
print(f"  Test samples: {(~train_mask).sum():,}")

# === Task 1: Anomaly Detection ===
TARGET_TASK1 = 'is_anomaly'

volatility_threshold = df.loc[train_mask, 'enrollment_volatility'].quantile(0.95)
print(f"\nTask 1 - Anomaly Detection")
print(f"  Volatility threshold (training data only): {volatility_threshold:.6f}")

df[TARGET_TASK1] = (
    (abs(df['z_score_rolling']) > 2) |
    (abs(df['z_score_state']) > 2.5) |
    (df['enrollment_volatility'] > volatility_threshold)
).astype(int)

print(f"  Overall anomaly rate: {df[TARGET_TASK1].mean():.2%}")
print(f"  Train anomaly rate: {df.loc[train_mask, TARGET_TASK1].mean():.2%}")
print(f"  Test anomaly rate: {df.loc[~train_mask, TARGET_TASK1].mean():.2%}")

# === Task 2: 7-Day Forecasting ===
TARGET_TASK2 = 'target_7d'
df[TARGET_TASK2] = df.groupby(['state', 'district', 'pincode'])['total_enrollments'].shift(-7)

print(f"\nTask 2 - 7-Day Forecasting")
print(f"  Target variable: Predict total_enrollments 7 days ahead")
print(f"  NaN rows (last 7 days): {df[TARGET_TASK2].isna().sum():,}")

# === Task 3: Spatial Inequality ===
TARGET_TASK3 = 'high_inequality'

threshold_inequality = df.loc[train_mask, 'z_score_state'].quantile(0.90)
df[TARGET_TASK3] = (df['z_score_state'] > threshold_inequality).astype(int)

print(f"\nTask 3 - Spatial Inequality")
print(f"  Z-score threshold (training data only): {threshold_inequality:.4f}")
print(f"  Overall inequality rate: {df[TARGET_TASK3].mean():.2%}")
print(f"  Train inequality rate: {df.loc[train_mask, TARGET_TASK3].mean():.2%}")
print(f"  Test inequality rate: {df.loc[~train_mask, TARGET_TASK3].mean():.2%}")

# --- Handle NaNs from Target Creation ---
df_task2 = df.dropna(subset=[TARGET_TASK2])
df_task1 = df.copy()
df_task3 = df.copy()

df_task3 = df.replace([np.inf, -np.inf], np.nan)

print(f"\nDataset Sizes:")
print(f"  Task 1/3: {df_task1.shape[0]:,} samples")
print(f"  Task 2:   {df_task2.shape[0]:,} samples (after dropping NaN targets)")


CREATING TARGET VARIABLES (LEAK-FREE)
Train/Test cutoff date: 2025-09-25
  Training samples: 179,423
  Test samples: 39,668

Task 1 - Anomaly Detection
  Volatility threshold (training data only): 1.168708
  Overall anomaly rate: 7.91%
  Train anomaly rate: 7.83%
  Test anomaly rate: 8.29%

Task 2 - 7-Day Forecasting
  Target variable: Predict total_enrollments 7 days ahead
  NaN rows (last 7 days): 126,781

Task 3 - Spatial Inequality
  Z-score threshold (training data only): 1.1365
  Overall inequality rate: 9.99%
  Train inequality rate: 10.00%
  Test inequality rate: 9.94%

Dataset Sizes:
  Task 1/3: 219,091 samples
  Task 2:   92,310 samples (after dropping NaN targets)


In [15]:
# Cell 11: Define Feature Sets (Leak-Free)
print("\n" + "="*60)
print("DEFINING FEATURE SETS (LEAK-FREE)")
print("="*60)

ALL_FEATURES = list(df.columns)
ID_COLS = ['state', 'district', 'pincode', 'date']
TARGET_COLS = [TARGET_TASK1, TARGET_TASK2, TARGET_TASK3]
RAW_VALUE_COLS = ['age_0_5', 'age_5_17', 'age_18_greater']
CATEGORICAL_COLS = ['state', 'district', 'pincode']

FREQUENCY_COLS = [col for col in ALL_FEATURES if '_frequency' in col]

# === Task 1: Anomaly Detection ===
LEAKING_COLS_TASK1 = [
    'z_score_rolling',
    'is_spike',
    'z_score_state',
    'enrollment_volatility',
    'state_total', 'state_mean', 'state_std', 'state_count',
    'district_total', 'district_mean', 'district_count',
    'pincode_vs_state_ratio',
    'pincode_vs_district_ratio',
    'district_vs_state_ratio',
    'total_enrollments',
]
LEAKING_COLS_TASK1.extend([col for col in ALL_FEATURES if '_pct_change_' in col])

FINAL_FEATURES_TASK1 = [
    col for col in ALL_FEATURES 
    if col not in ID_COLS + TARGET_COLS + RAW_VALUE_COLS + LEAKING_COLS_TASK1 + FREQUENCY_COLS
]
FINAL_FEATURES_TASK1.extend(CATEGORICAL_COLS)

print(f"\nTask 1 Features: {len(FINAL_FEATURES_TASK1)}")
print(f"  - Categorical: {len(CATEGORICAL_COLS)}")
print(f"  - Excluded: {len(LEAKING_COLS_TASK1)} leaking features")

# === Task 2: Forecasting (FIXED - No Duplicates) ===
LEAKING_COLS_TASK2 = [
    'state_total', 'state_mean', 'state_std', 'state_count',
    'district_total', 'district_mean', 'district_count',
    'pincode_vs_state_ratio', 'pincode_vs_district_ratio',
    'district_vs_state_ratio',
    'z_score_state', 'z_score_rolling', 'enrollment_volatility', 'is_spike'
]
LEAKING_COLS_TASK2.extend([col for col in ALL_FEATURES if '_pct_change_' in col])

FINAL_FEATURES_TASK2 = [
    col for col in ALL_FEATURES 
    if col not in ID_COLS + TARGET_COLS + RAW_VALUE_COLS + LEAKING_COLS_TASK2 + FREQUENCY_COLS
]
FINAL_FEATURES_TASK2.extend(CATEGORICAL_COLS)

# FIX: Only add total_enrollments if not already present
if 'total_enrollments' not in FINAL_FEATURES_TASK2:
    FINAL_FEATURES_TASK2.append('total_enrollments')

# Verify no duplicates
assert len(FINAL_FEATURES_TASK2) == len(set(FINAL_FEATURES_TASK2)), "Task 2 has duplicate features!"

print(f"\nTask 2 Features: {len(FINAL_FEATURES_TASK2)}")
print(f"  - Categorical: {len(CATEGORICAL_COLS)}")
print(f"  ✓ No duplicate columns")

# === Task 3: Spatial Inequality ===
LEAKING_COLS_TASK3 = [
    'state_total', 'state_mean', 'state_std', 'state_count',
    'district_total', 'district_mean', 'district_count',
    'z_score_state',
    'pincode_vs_state_ratio',
    'pincode_vs_district_ratio',
    'district_vs_state_ratio',
    'z_score_rolling', 'is_spike', 'enrollment_volatility',
    'total_enrollments'
]

FINAL_FEATURES_TASK3 = [
    col for col in ALL_FEATURES 
    if col not in ID_COLS + TARGET_COLS + RAW_VALUE_COLS + LEAKING_COLS_TASK3 + FREQUENCY_COLS
]
FINAL_FEATURES_TASK3.extend(CATEGORICAL_COLS)

print(f"\nTask 3 Features: {len(FINAL_FEATURES_TASK3)}")
print(f"  - Categorical: {len(CATEGORICAL_COLS)}")


DEFINING FEATURE SETS (LEAK-FREE)

Task 1 Features: 50
  - Categorical: 3
  - Excluded: 23 leaking features

Task 2 Features: 51
  - Categorical: 3
  ✓ No duplicate columns

Task 3 Features: 58
  - Categorical: 3


In [16]:
# Cell 12: Define CV Function
def run_time_series_cv(train_data, df_dates, task_name, label, eval_metric, problem_type, n_splits=3):
    """
    Perform time-series cross-validation with detailed reporting
    """
    print(f"\n{'='*60}")
    print(f"TIME-SERIES CV: {task_name}")
    print(f"{'='*60}")
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(tscv.split(train_data)):
        print(f"\nFold {fold_idx + 1}/{n_splits}")
        
        X_fold_train = train_data.iloc[train_idx]
        X_fold_val = train_data.iloc[val_idx]
        
        train_dates = df_dates.iloc[train_idx]
        val_dates = df_dates.iloc[val_idx]
        
        print(f"  Train: {len(X_fold_train):,} samples | {train_dates.min().date()} to {train_dates.max().date()}")
        print(f"  Val:   {len(X_fold_val):,} samples | {val_dates.min().date()} to {val_dates.max().date()}")
        
        predictor_cv = TabularPredictor(
            label=label,
            path=f"autogluon_models/{task_name}_cv_fold{fold_idx}",
            eval_metric=eval_metric,
            problem_type=problem_type,
            verbosity=0
        )
        
        predictor_cv.fit(
            X_fold_train,
            presets='medium_quality',
            time_limit=600,
            ag_args_fit={'num_gpus': 0}
        )
        
        # Get the score - evaluate returns a dict, extract the metric value
        score_dict = predictor_cv.evaluate(X_fold_val, silent=True)
        if isinstance(score_dict, dict):
            score = score_dict.get(eval_metric, list(score_dict.values())[0])
        else:
            score = score_dict
        cv_scores.append(score)
        
        print(f"  Fold {fold_idx + 1} {eval_metric}: {score:.4f}")
        
        predictor_cv.save_space()
    
    print(f"\n{task_name} CV Results:")
    print(f"  Mean {eval_metric}: {np.mean(cv_scores):.4f} +/- {np.std(cv_scores):.4f}")
    print(f"  Min: {np.min(cv_scores):.4f} | Max: {np.max(cv_scores):.4f}")
    
    return cv_scores

print("✓ CV function defined")

✓ CV function defined


In [13]:
# Cell 13: Task 1 - Anomaly Detection (CV + Training)
print("\n" + "="*60)
print("TASK 1: ANOMALY DETECTION")
print("="*60)

train_data_task1 = df_task1[FINAL_FEATURES_TASK1 + [TARGET_TASK1]].copy()

for cat_col in CATEGORICAL_COLS:
    train_data_task1[cat_col] = train_data_task1[cat_col].astype('category')

print(f"\nPre-split class distribution:")
print(train_data_task1[TARGET_TASK1].value_counts())
if train_data_task1[TARGET_TASK1].nunique() < 2:
    raise ValueError("Only one class present in Task 1!")

X_train_t1 = train_data_task1[df_task1['date'] <= TRAIN_TEST_CUTOFF]
X_test_t1 = train_data_task1[df_task1['date'] > TRAIN_TEST_CUTOFF]

print(f"\nTime-based split:")
print(f"  Train: {len(X_train_t1):,} samples (up to {TRAIN_TEST_CUTOFF.date()})")
print(f"  Test:  {len(X_test_t1):,} samples (after {TRAIN_TEST_CUTOFF.date()})")
print(f"  Train anomaly rate: {X_train_t1[TARGET_TASK1].mean():.2%}")
print(f"  Test anomaly rate:  {X_test_t1[TARGET_TASK1].mean():.2%}")

# Run CV
cv_scores_t1 = run_time_series_cv(
    X_train_t1,
    df_task1.loc[X_train_t1.index, 'date'],
    'task1_anomaly',
    TARGET_TASK1,
    'roc_auc',
    'binary',
    n_splits=3
)


TASK 1: ANOMALY DETECTION

Pre-split class distribution:
is_anomaly
0    201756
1     17335
Name: count, dtype: int64

Time-based split:
  Train: 179,423 samples (up to 2025-09-25)
  Test:  39,668 samples (after 2025-09-25)
  Train anomaly rate: 7.83%
  Test anomaly rate:  8.29%

TIME-SERIES CV: task1_anomaly

Fold 1/3
  Train: 44,858 samples | 2025-03-09 to 2025-09-25
  Val:   44,855 samples | 2025-03-09 to 2025-09-25
  Fold 1 roc_auc: 0.9734

Fold 2/3
  Train: 89,713 samples | 2025-03-09 to 2025-09-25
  Val:   44,855 samples | 2025-03-02 to 2025-09-25
  Fold 2 roc_auc: 0.9835

Fold 3/3
  Train: 134,568 samples | 2025-03-02 to 2025-09-25
  Val:   44,855 samples | 2025-03-09 to 2025-09-25
  Fold 3 roc_auc: 0.9891

task1_anomaly CV Results:
  Mean roc_auc: 0.9820 +/- 0.0065
  Min: 0.9734 | Max: 0.9891


In [14]:
# Cell 14: Task 1 - Final Model Training
predictor_task1 = TabularPredictor(
    label=TARGET_TASK1,
    path="autogluon_models/task1_anomaly_PRODUCTION",
    eval_metric='roc_auc',
    problem_type='binary'
)

print(f"\nTraining final Task 1 model (30 min limit)...")
predictor_task1.fit(
    X_train_t1,
    presets='best_quality',
    time_limit=1800,
    ag_args_fit={'num_gpus': 0}
)

print(f"\nTask 1 Final Results:")
leaderboard_t1 = predictor_task1.leaderboard(X_test_t1, silent=True)
print("\nLeaderboard (Top 5 models):")
print(leaderboard_t1[['model', 'score_test', 'score_val', 'pred_time_test', 'fit_time']].head())

eval_t1_dict = predictor_task1.evaluate(X_test_t1, silent=True)
eval_t1 = eval_t1_dict.get('roc_auc', list(eval_t1_dict.values())[0]) if isinstance(eval_t1_dict, dict) else eval_t1_dict
print(f"\nTest ROC-AUC: {eval_t1:.4f}")
print(f"CV ROC-AUC: {np.mean(cv_scores_t1):.4f} +/- {np.std(cv_scores_t1):.4f}")

predictor_task1.save_space()

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Oct 23 15:35:13 UTC 2025
CPU Count:          12
Memory Avail:       5.99 GB / 15.25 GB (39.3%)
Disk Space Avail:   423.65 GB / 464.17 GB (91.3%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the 


Training final Task 1 model (30 min limit)...


2025-11-02 20:56:29,515	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/home/vulcan/Abhay/Projects/ADA/Baseline-2/autogluon_models/task1_anomaly_PRODUCTION/ds_sub_fit/sub_fit_ho"
(_dystack pid=125455) Running DyStack sub-fit ...
(_dystack pid=125455) Beginning AutoGluon training ... Time limit = 448s
(_dystack pid=125455) AutoGluon will save models to "/home/vulcan/Abhay/Projects/ADA/Baseline-2/autogluon_models/task1_anomaly_PRODUCTION/ds_sub_fit/sub_fit_ho"
(_dystack pid=125455) Train Data Rows:    159487
(_dystack pid=125455) Train Data Columns: 50
(_dystack pid=125455) Label Column:       is_anomaly
(_dystack pid=125455) Problem Type:       binary
(_dystack pid=125455) Preprocessing data ...
(_dystack pid=125455) Selected class <--> label mapping:  class 1 = 1, class 0 = 0
(_dystack pid=125455) Using Feature Generators to preprocess the data ...
(_dystack pid=125455) Fitting AutoMLPipelineFeatureGenerator...
(_dysta

(_ray_fit pid=126238) [1000]	valid_set's binary_logloss: 0.0456411
(_ray_fit pid=126241) [1000]	valid_set's binary_logloss: 0.0490498 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=126239) [1000]	valid_set's binary_logloss: 0.0476152 [repeated 4x across cluster]
(_ray_fit pid=126238) [2000]	valid_set's binary_logloss: 0.0435471
(_ray_fit pid=126242) [2000]	valid_set's binary_logloss: 0.0452567
(_ray_fit pid=126236) [2000]	valid_set's binary_logloss: 0.042709
(_ray_fit pid=126240) [2000]	valid_set's binary_logloss: 0.0436297
(_ray_fit pid=126239) [2000]	valid_set's binary_logloss: 0.0457564 [repeated 4x across cluster]


(_dystack pid=125455) 	0.9945	 = Validation score   (roc_auc)
(_dystack pid=125455) 	90.77s	 = Training   runtime
(_dystack pid=125455) 	58.22s	 = Validation runtime
(_dystack pid=125455) Fitting model: LightGBM_BAG_L1 ... Training model for up to 198.50s of the 347.48s of remaining time.
(_dystack pid=125455) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=7.29%)


(_ray_fit pid=126972) [1000]	valid_set's binary_logloss: 0.0472208
(_ray_fit pid=126974) [1000]	valid_set's binary_logloss: 0.0483599
(_ray_fit pid=126971) [1000]	valid_set's binary_logloss: 0.0460921 [repeated 2x across cluster]
(_ray_fit pid=126972) [2000]	valid_set's binary_logloss: 0.0465112 [repeated 4x across cluster]
(_ray_fit pid=126971) [2000]	valid_set's binary_logloss: 0.0451534 [repeated 2x across cluster]
(_ray_fit pid=126972) [3000]	valid_set's binary_logloss: 0.0460778 [repeated 2x across cluster]
(_ray_fit pid=126971) [3000]	valid_set's binary_logloss: 0.0458322
(_ray_fit pid=126972) [4000]	valid_set's binary_logloss: 0.047536


(_dystack pid=125455) 	0.9943	 = Validation score   (roc_auc)
(_dystack pid=125455) 	91.67s	 = Training   runtime
(_dystack pid=125455) 	39.44s	 = Validation runtime
(_dystack pid=125455) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 98.87s of the 247.85s of remaining time.
(_dystack pid=125455) 	0.9926	 = Validation score   (roc_auc)
(_dystack pid=125455) 	17.06s	 = Training   runtime
(_dystack pid=125455) 	3.5s	 = Validation runtime
(_dystack pid=125455) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 78.05s of the 227.03s of remaining time.
(_dystack pid=125455) 	0.9931	 = Validation score   (roc_auc)
(_dystack pid=125455) 	13.22s	 = Training   runtime
(_dystack pid=125455) 	3.37s	 = Validation runtime
(_dystack pid=125455) Fitting model: CatBoost_BAG_L1 ... Training model for up to 61.24s of the 210.22s of remaining time.
(_dystack pid=125455) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 1


Task 1 Final Results:

Leaderboard (Top 5 models):
                 model  score_test  score_val  pred_time_test     fit_time
0    LightGBMXT_BAG_L2    0.968810   0.997069       35.352586   854.072739
1  WeightedEnsemble_L2    0.967313   0.996736       30.283732   740.356148
2      CatBoost_BAG_L2    0.967146   0.997796       35.228734  1132.165778
3      LightGBM_BAG_L2    0.965759   0.995754       35.039743   849.718428
4      CatBoost_BAG_L1    0.965758   0.996169        0.686730   496.237592

Test ROC-AUC: 0.9652
CV ROC-AUC: 0.9820 +/- 0.0065


In [17]:
# Cell 15: Task 2 - 7-Day Forecasting (CV + Training) - FIXED
print("\n" + "="*60)
print("TASK 2: 7-DAY FORECASTING")
print("="*60)

# FIX: Remove any potential duplicates in feature list before creating training data
FINAL_FEATURES_TASK2_UNIQUE = list(dict.fromkeys(FINAL_FEATURES_TASK2))  # Preserves order, removes dupes
print(f"Task 2 features: {len(FINAL_FEATURES_TASK2_UNIQUE)} (deduplicated)")

train_data_task2 = df_task2[FINAL_FEATURES_TASK2_UNIQUE + [TARGET_TASK2]].copy()

for cat_col in CATEGORICAL_COLS:
    if cat_col in train_data_task2.columns:
        train_data_task2[cat_col] = train_data_task2[cat_col].astype('category')

cutoff_date_t2 = df_task2['date'].quantile(0.8)
X_train_t2 = train_data_task2[df_task2['date'] <= cutoff_date_t2]
X_test_t2 = train_data_task2[df_task2['date'] > cutoff_date_t2]

print(f"\nTime-based split:")
print(f"  Train: {len(X_train_t2):,} samples (up to {cutoff_date_t2.date()})")
print(f"  Test:  {len(X_test_t2):,} samples (after {cutoff_date_t2.date()})")

# Verify no duplicate columns
assert X_train_t2.columns.duplicated().sum() == 0, "X_train_t2 has duplicate columns!"
print("✓ No duplicate columns in training data")

# Run CV
cv_scores_t2 = run_time_series_cv(
    X_train_t2,
    df_task2.loc[X_train_t2.index, 'date'],
    'task2_forecasting',
    TARGET_TASK2,
    'root_mean_squared_error',
    'regression',
    n_splits=3
)


TASK 2: 7-DAY FORECASTING
Task 2 features: 51 (deduplicated)

Time-based split:
  Train: 74,339 samples (up to 2025-09-15)
  Test:  17,971 samples (after 2025-09-15)
✓ No duplicate columns in training data

TIME-SERIES CV: task2_forecasting

Fold 1/3
  Train: 18,587 samples | 2025-03-09 to 2025-09-15
  Val:   18,584 samples | 2025-03-09 to 2025-09-15
  Fold 1 root_mean_squared_error: -4.7478

Fold 2/3
  Train: 37,171 samples | 2025-03-09 to 2025-09-15
  Val:   18,584 samples | 2025-03-02 to 2025-09-15
  Fold 2 root_mean_squared_error: -25.0665

Fold 3/3
  Train: 55,755 samples | 2025-03-02 to 2025-09-15
  Val:   18,584 samples | 2025-03-09 to 2025-09-15
  Fold 3 root_mean_squared_error: -4.4301

task2_forecasting CV Results:
  Mean root_mean_squared_error: -11.4148 +/- 9.6541
  Min: -25.0665 | Max: -4.4301


In [18]:
# Diagnostic: Check Task 2 Target
print("Task 2 Target Diagnostics:")
print(f"Target column: {TARGET_TASK2}")
print(f"Data type: {X_train_t2[TARGET_TASK2].dtype}")
print(f"Unique values: {X_train_t2[TARGET_TASK2].nunique()}")
print(f"Min: {X_train_t2[TARGET_TASK2].min()}")
print(f"Max: {X_train_t2[TARGET_TASK2].max()}")
print(f"Mean: {X_train_t2[TARGET_TASK2].mean():.2f}")
print(f"\nFirst 10 values:")
print(X_train_t2[TARGET_TASK2].head(10))

# Cell 16: Task 2 - Final Model Training (FIXED)
predictor_task2 = TabularPredictor(
    label=TARGET_TASK2,
    path="autogluon_models/task2_forecasting_PRODUCTION",
    eval_metric='root_mean_squared_error',
    problem_type='regression'  # ← FIX: Force regression
)

print(f"\nTraining final Task 2 model (30 min limit)...")
predictor_task2.fit(
    X_train_t2,
    presets='best_quality',
    time_limit=1800,
    ag_args_fit={'num_gpus': 0}
)

print(f"\nTask 2 Final Results:")
leaderboard_t2 = predictor_task2.leaderboard(X_test_t2, silent=True)
print("\nLeaderboard (Top 5 models):")
print(leaderboard_t2[['model', 'score_test', 'score_val', 'pred_time_test', 'fit_time']].head())

eval_t2_dict = predictor_task2.evaluate(X_test_t2, silent=True)
eval_t2 = eval_t2_dict.get('root_mean_squared_error', list(eval_t2_dict.values())[0]) if isinstance(eval_t2_dict, dict) else eval_t2_dict
print(f"\nTest RMSE: {eval_t2:.2f}")
print(f"CV RMSE: {np.mean(cv_scores_t2):.2f} +/- {np.std(cv_scores_t2):.2f}")

predictor_task2.save_space()

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Oct 23 15:35:13 UTC 2025
CPU Count:          12
Memory Avail:       3.90 GB / 15.25 GB (25.6%)
Disk Space Avail:   423.06 GB / 464.17 GB (91.1%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the 

Task 2 Target Diagnostics:
Target column: target_7d
Data type: float64
Unique values: 88
Min: 1.0
Max: 2538.0
Mean: 4.24

First 10 values:
0     1.0
1     1.0
2     1.0
3     1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
Name: target_7d, dtype: float64

Training final Task 2 model (30 min limit)...


Leaderboard on holdout data (DyStack):
                     model  score_holdout  score_val              eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3      -4.015254 -11.093464  root_mean_squared_error        5.446167      14.923217  350.780691                 0.001598                0.001001           0.054681            3       True         12
1          CatBoost_BAG_L1      -4.042099 -13.886574  root_mean_squared_error        0.412622       0.282004  166.524093                 0.412622                0.282004         166.524093            1       True          4
2          CatBoost_BAG_L2      -4.054214 -13.621659  root_mean_squared_error        4.384526       9.926616  299.874202                 0.075618                0.174602          24.109327            2       True         11
3   RandomForestMSE_BAG_L2      -4.066184 -12.049713  root_mean_s


Task 2 Final Results:

Leaderboard (Top 5 models):
                    model  score_test  score_val  pred_time_test    fit_time
0   NeuralNetTorch_BAG_L1    -5.22716  -9.084648        0.715967  161.993685
1     WeightedEnsemble_L3    -5.22716  -9.084648        0.717202  162.124785
2     WeightedEnsemble_L2    -5.22716  -9.084648        0.717401  162.042589
3  NeuralNetFastAI_BAG_L1    -5.25384 -11.255325        1.357575  355.331522
4         LightGBM_BAG_L2    -7.18113 -13.054572        7.019010  874.174177

Test RMSE: -5.23
CV RMSE: -11.41 +/- 9.65


In [17]:
# Cell 17: Task 3 - Spatial Inequality (CV + Training)
print("\n" + "="*60)
print("TASK 3: SPATIAL INEQUALITY")
print("="*60)

train_data_task3 = df_task3[FINAL_FEATURES_TASK3 + [TARGET_TASK3]].copy()

for cat_col in CATEGORICAL_COLS:
    train_data_task3[cat_col] = train_data_task3[cat_col].astype('category')

print(f"\nPre-split class distribution:")
print(train_data_task3[TARGET_TASK3].value_counts())
if train_data_task3[TARGET_TASK3].nunique() < 2:
    raise ValueError("Only one class present in Task 3!")

X_train_t3 = train_data_task3[df_task3['date'] <= TRAIN_TEST_CUTOFF]
X_test_t3 = train_data_task3[df_task3['date'] > TRAIN_TEST_CUTOFF]

print(f"\nTime-based split:")
print(f"  Train: {len(X_train_t3):,} samples")
print(f"  Test:  {len(X_test_t3):,} samples")
print(f"  Train inequality rate: {X_train_t3[TARGET_TASK3].mean():.2%}")
print(f"  Test inequality rate:  {X_test_t3[TARGET_TASK3].mean():.2%}")

# Run CV
cv_scores_t3 = run_time_series_cv(
    X_train_t3,
    df_task3.loc[X_train_t3.index, 'date'],
    'task3_inequality',
    TARGET_TASK3,
    'roc_auc',
    'binary',
    n_splits=3
)


TASK 3: SPATIAL INEQUALITY

Pre-split class distribution:
high_inequality
0    197213
1     21878
Name: count, dtype: int64

Time-based split:
  Train: 179,423 samples
  Test:  39,668 samples
  Train inequality rate: 10.00%
  Test inequality rate:  9.94%

TIME-SERIES CV: task3_inequality

Fold 1/3
  Train: 44,858 samples | 2025-03-09 to 2025-09-25
  Val:   44,855 samples | 2025-03-09 to 2025-09-25
  Fold 1 roc_auc: 0.9405

Fold 2/3
  Train: 89,713 samples | 2025-03-09 to 2025-09-25
  Val:   44,855 samples | 2025-03-02 to 2025-09-25
  Fold 2 roc_auc: 0.9586

Fold 3/3
  Train: 134,568 samples | 2025-03-02 to 2025-09-25
  Val:   44,855 samples | 2025-03-09 to 2025-09-25
  Fold 3 roc_auc: 0.9620

task3_inequality CV Results:
  Mean roc_auc: 0.9537 +/- 0.0094
  Min: 0.9405 | Max: 0.9620


In [19]:
# Cell 18: Task 3 - Final Model Training
predictor_task3 = TabularPredictor(
    label=TARGET_TASK3,
    path="autogluon_models/task3_inequality_PRODUCTION",
    eval_metric='roc_auc',
    problem_type='binary'
)

print(f"\nTraining final Task 3 model (30 min limit)...")
predictor_task3.fit(
    X_train_t3,
    presets='best_quality',
    time_limit=1800,
    ag_args_fit={'num_gpus': 0}
)

print(f"\nTask 3 Final Results:")
leaderboard_t3 = predictor_task3.leaderboard(X_test_t3, silent=True)
print("\nLeaderboard (Top 5 models):")
print(leaderboard_t3[['model', 'score_test', 'score_val', 'pred_time_test', 'fit_time']].head())

eval_t3_dict = predictor_task3.evaluate(X_test_t3, silent=True)
eval_t3 = eval_t3_dict.get('roc_auc', list(eval_t3_dict.values())[0]) if isinstance(eval_t3_dict, dict) else eval_t3_dict
print(f"\nTest ROC-AUC: {eval_t3:.4f}")
print(f"CV ROC-AUC: {np.mean(cv_scores_t3):.4f} +/- {np.std(cv_scores_t3):.4f}")

predictor_task3.save_space()

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Oct 23 15:35:13 UTC 2025
CPU Count:          12
Memory Avail:       6.96 GB / 15.25 GB (45.7%)
Disk Space Avail:   422.87 GB / 464.17 GB (91.1%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the 


Training final Task 3 model (30 min limit)...


Leaderboard on holdout data (DyStack):
                     model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        LightGBMXT_BAG_L2       0.999137   0.998705     roc_auc       14.345190     328.377805  282.658589                 0.380523                2.416536          16.888775            2       True          4
1      WeightedEnsemble_L3       0.999073   0.999006     roc_auc       14.687805     335.003201  368.979917                 0.001913                0.016499           3.268507            3       True         10
2          CatBoost_BAG_L2       0.999031   0.998966     roc_auc       14.286044     326.393426  331.433492                 0.321377                0.432157          65.663678            2       True          8
3    ExtraTreesGini_BAG_L2       0.998992   0.998120     roc_auc       14.064697     328.254058  273.169366          


Task 3 Final Results:

Leaderboard (Top 5 models):
                 model  score_test  score_val  pred_time_test     fit_time
0      CatBoost_BAG_L1    0.954865   0.997697        0.754276   363.606946
1      CatBoost_BAG_L2    0.951403   0.999259       61.655612  1122.977340
2    LightGBMXT_BAG_L1    0.950873   0.998669       25.310220   159.780903
3  WeightedEnsemble_L2    0.949935   0.998974       55.135283   365.093319
4      LightGBM_BAG_L1    0.947383   0.998894       29.823109   201.160483

Test ROC-AUC: 0.9285
CV ROC-AUC: 0.9537 +/- 0.0094


In [21]:
# Cell 19: Final Summary
print("\n" + "="*60)
print("PRODUCTION PIPELINE COMPLETE")
print("="*60)
print("\nData Leak Prevention:")
print("  ✓ Task 1: Volatility threshold from training data only")
print("  ✓ Task 1: Excluded ALL ratio features derived from aggregates")
print("  ✓ Task 2: Date-based split + contemporaneous total_enrollments")
print("  ✓ Task 3: Inequality threshold from training data only")
print("  ✓ NaN imputation: AutoGluon handles naturally")
print("  ✓ All splits: Strictly time-based")
print("\nFinal Scores:")
print(f"  Task 1 (Anomaly Detection):")
print(f"    Test ROC-AUC: {eval_t1:.4f}")
print(f"    CV ROC-AUC: {np.mean(cv_scores_t1):.4f} +/- {np.std(cv_scores_t1):.4f}")
print(f"  Task 2 (7-Day Forecasting):")
print(f"    Test RMSE: {eval_t2:.2f}")
print(f"    CV RMSE: {np.mean(cv_scores_t2):.2f} +/- {np.std(cv_scores_t2):.2f}")
print(f"  Task 3 (Spatial Inequality):")
print(f"    Test ROC-AUC: {eval_t3:.4f}")
print(f"    CV ROC-AUC: {np.mean(cv_scores_t3):.4f} +/- {np.std(cv_scores_t3):.4f}")

print("\n" + "="*60)
print("✓ LEAK-FREE BASELINE COMPLETE")
print("="*60)
print("Models saved to: autogluon_models/task*_PRODUCTION/")


PRODUCTION PIPELINE COMPLETE

Data Leak Prevention:
  ✓ Task 1: Volatility threshold from training data only
  ✓ Task 1: Excluded ALL ratio features derived from aggregates
  ✓ Task 2: Date-based split + contemporaneous total_enrollments
  ✓ Task 3: Inequality threshold from training data only
  ✓ NaN imputation: AutoGluon handles naturally
  ✓ All splits: Strictly time-based

Final Scores:
  Task 1 (Anomaly Detection):
    Test ROC-AUC: 0.9652
    CV ROC-AUC: 0.9820 +/- 0.0065
  Task 2 (7-Day Forecasting):
    Test RMSE: -5.23
    CV RMSE: -11.41 +/- 9.65
  Task 3 (Spatial Inequality):
    Test ROC-AUC: 0.9485
    CV ROC-AUC: 0.9536 +/- 0.0094

✓ LEAK-FREE BASELINE COMPLETE
Models saved to: autogluon_models/task*_PRODUCTION/


In [22]:
# Cell 20: Generate Prediction CSV Files
print("\n" + "="*60)
print("GENERATING PREDICTION CSV FILES")
print("="*60)

# === Task 1: Anomaly Detection Predictions ===
print("\nTask 1 - Anomaly Detection:")
y_pred_t1 = predictor_task1.predict(X_test_t1)
y_pred_proba_t1 = predictor_task1.predict_proba(X_test_t1)

# Get test indices and merge with original data
test_indices_t1 = X_test_t1.index
task1_results = pd.DataFrame({
    'date': df_task1.loc[test_indices_t1, 'date'],
    'state': df_task1.loc[test_indices_t1, 'state'],
    'district': df_task1.loc[test_indices_t1, 'district'],
    'pincode': df_task1.loc[test_indices_t1, 'pincode'],
    'actual_is_anomaly': X_test_t1[TARGET_TASK1],
    'predicted_is_anomaly': y_pred_t1,
    'anomaly_probability': y_pred_proba_t1[1] if len(y_pred_proba_t1.shape) > 1 else y_pred_proba_t1
})

task1_results.to_csv('task1_anomaly_predictions.csv', index=False)
print(f"✓ Saved: task1_anomaly_predictions.csv ({len(task1_results):,} rows)")
print(f"  Accuracy: {(task1_results['actual_is_anomaly'] == task1_results['predicted_is_anomaly']).mean():.2%}")

# === Task 2: 7-Day Forecasting Predictions ===
print("\nTask 2 - 7-Day Forecasting:")
y_pred_t2 = predictor_task2.predict(X_test_t2)

test_indices_t2 = X_test_t2.index
task2_results = pd.DataFrame({
    'date': df_task2.loc[test_indices_t2, 'date'],
    'state': df_task2.loc[test_indices_t2, 'state'],
    'district': df_task2.loc[test_indices_t2, 'district'],
    'pincode': df_task2.loc[test_indices_t2, 'pincode'],
    'actual_enrollments_7d': X_test_t2[TARGET_TASK2],
    'predicted_enrollments_7d': y_pred_t2,
    'absolute_error': abs(X_test_t2[TARGET_TASK2] - y_pred_t2),
    'percentage_error': abs((X_test_t2[TARGET_TASK2] - y_pred_t2) / (X_test_t2[TARGET_TASK2] + 1e-10)) * 100
})

task2_results.to_csv('task2_forecasting_predictions.csv', index=False)
print(f"✓ Saved: task2_forecasting_predictions.csv ({len(task2_results):,} rows)")
print(f"  RMSE: {np.sqrt(np.mean((task2_results['actual_enrollments_7d'] - task2_results['predicted_enrollments_7d'])**2)):.2f}")
print(f"  MAE: {task2_results['absolute_error'].mean():.2f}")
print(f"  MAPE: {task2_results['percentage_error'].mean():.2f}%")

# === Task 3: Spatial Inequality Predictions ===
print("\nTask 3 - Spatial Inequality:")
y_pred_t3 = predictor_task3.predict(X_test_t3)
y_pred_proba_t3 = predictor_task3.predict_proba(X_test_t3)

test_indices_t3 = X_test_t3.index
task3_results = pd.DataFrame({
    'date': df_task3.loc[test_indices_t3, 'date'],
    'state': df_task3.loc[test_indices_t3, 'state'],
    'district': df_task3.loc[test_indices_t3, 'district'],
    'pincode': df_task3.loc[test_indices_t3, 'pincode'],
    'actual_high_inequality': X_test_t3[TARGET_TASK3],
    'predicted_high_inequality': y_pred_t3,
    'inequality_probability': y_pred_proba_t3[1] if len(y_pred_proba_t3.shape) > 1 else y_pred_proba_t3
})

task3_results.to_csv('task3_inequality_predictions.csv', index=False)
print(f"✓ Saved: task3_inequality_predictions.csv ({len(task3_results):,} rows)")
print(f"  Accuracy: {(task3_results['actual_high_inequality'] == task3_results['predicted_high_inequality']).mean():.2%}")

# === Best Model Information ===
print("\n" + "="*60)
print("BEST MODELS USED:")
print("="*60)
print(f"\nTask 1 Best Model: {predictor_task1.get_model_best()}")
print(f"Task 2 Best Model: {predictor_task2.get_model_best()}")
print(f"Task 3 Best Model: {predictor_task3.get_model_best()}")

print("\n" + "="*60)
print("✓ ALL PREDICTION CSV FILES SAVED")
print("="*60)
print("Files created:")
print("  - task1_anomaly_predictions.csv")
print("  - task2_forecasting_predictions.csv")
print("  - task3_inequality_predictions.csv")


GENERATING PREDICTION CSV FILES

Task 1 - Anomaly Detection:
✓ Saved: task1_anomaly_predictions.csv (39,668 rows)
  Accuracy: 96.37%

Task 2 - 7-Day Forecasting:
✓ Saved: task2_forecasting_predictions.csv (17,971 rows)
  RMSE: 5.23
  MAE: 2.73
  MAPE: 88.50%

Task 3 - Spatial Inequality:
✓ Saved: task3_inequality_predictions.csv (39,668 rows)
  Accuracy: 93.16%

BEST MODELS USED:


AttributeError: 'TabularPredictor' object has no attribute 'get_model_best'